## Helper Pretend Functions

In [ ]:
import numpy
import random
import math
import time

num_joints = 6

def trajectory_sample():
    q_rand = []
    for i in range(num_joints):
        q_rand.append(random.uniform(-math.pi, math.pi))
    return q_rand


def find_closest_point_in_tree(tree, r):
        shortest_distance = numpy.linalg.norm(r-tree[0].q)
        closest_point = tree[0]
        for i in range(1, len(tree)-1):
            if shortest_distance > numpy.linalg.norm(r-tree[i].q):
                shortest_distance = numpy.linalg.norm(r-tree[i].q)
                closest_point = tree[i]
        return closest_point


def step(q_near, q_rand, step_size=0.1):
        direction = [qr - qn for qr, qn in zip(q_rand, q_near)]
        length = math.sqrt(sum([d**2 for d in direction]))
        if length == 0:
            return q_near
        step = [qn + (d / length) * step_size for qn, d in zip(q_near, direction)]
        return [max(min(q, math.pi), -math.pi) for q in step]

def is_segment_valid():
  num = numpy.random.rand(1)
  if num < 0.05:
    return True
  return False

class RRTBranch(object):
    def __init__(self, parent, q):
        self.parent = parent
        self.q = q


### Serial Sampling Python

In [ ]:
q_current = trajectory_sample()
q_goal = trajectory_sample()
max_iterations = 1000
tree = [RRTBranch(parent=None, q=q_current)]



def find_path(q):
    #this is the portion of motion planning that
    for i in range(max_iterations):
            q_rand = trajectory_sample()
            nearest_point = find_closest_point_in_tree(tree, numpy.array(q_rand))

            #step towards the random configuration
            q_new = step(nearest_point.q, q_rand)

            if is_segment_valid():
                #add to tree and check if possible to get to goal
                new_branch = RRTBranch(parent=nearest_point, q=q_new)
                tree.append(new_branch)

                if is_segment_valid():
                    goal_branch = RRTBranch(parent=new_branch, q=q_goal)
                    tree.append(goal_branch)
                    found = True
                    amount_to_sleep = numpy.random.rand(1)[0] * 1
                    time.sleep(amount_to_sleep)
                    print(amount_to_sleep, tree)
                    q.put(tree)
                    return tree

    raise RuntimeError("Path to goal could not be found.")


In [ ]:
tree = find_path()

TypeError: find_path() missing 1 required positional argument: 'q'

### MultiTherading version one

this one runs until the top m threads are done. These put their result in the queue which I print in results. Step one just pick the first thread that ends, but ideally you would pick the best thread path out of the best n

In [ ]:
import threading
from multiprocessing import Pool
import multiprocessing

import queue

def find_path(q):
    #this is the portion of motion planning that
    for i in range(max_iterations):
            q_rand = []
            for i in range(num_joints):
                q_rand.append(random.uniform(-math.pi, math.pi))
            nearest_point = find_closest_point_in_tree(tree, numpy.array(q_rand))

            #step towards the random configuration
            q_new = step(nearest_point.q, q_rand)

            num = numpy.random.rand(1)
                if num < 0.05:
                #add to tree and check if possible to get to goal
                new_branch = RRTBranch(parent=nearest_point, q=q_new)
                tree.append(new_branch)

                goal_branch = RRTBranch(parent=new_branch, q=q_goal)
                tree.append(goal_branch)
                found = True
                q.put(tree)
                return tree


q = queue.Queue()
n=10 #number of threads
m=3 #Top X paths to return, set this one 1 initally and if you put everything into the smoother you could return the best part out of these three as well

threads = []

for i in range(n):
    t = threading.Thread(target=find_path, args=(q,))
    t.start()
    threads.append(t)

current_threads= threading.active_count()
amount_to_wait_for = current_threads-m


while threading.active_count() > amount_to_wait_for:
  time.sleep(0.000000000001)



results = []
while not q.empty():
    results.append(q.get())

len(results)


0.010001731926091328 [<__main__.RRTBranch object at 0x7b4f9867a440>, <__main__.RRTBranch object at 0x7b4f98678d90>, <__main__.RRTBranch object at 0x7b4f9867b0a0>, <__main__.RRTBranch object at 0x7b4f9867ae60>, <__main__.RRTBranch object at 0x7b4f9867aec0>, <__main__.RRTBranch object at 0x7b4f9867b160>, <__main__.RRTBranch object at 0x7b4f9867b100>, <__main__.RRTBranch object at 0x7b4f9867b580>, <__main__.RRTBranch object at 0x7b4f986784f0>, <__main__.RRTBranch object at 0x7b4f9867a590>, <__main__.RRTBranch object at 0x7b4f9867b850>, <__main__.RRTBranch object at 0x7b4f9867a530>, <__main__.RRTBranch object at 0x7b4f98679630>, <__main__.RRTBranch object at 0x7b4f98678850>, <__main__.RRTBranch object at 0x7b4f9867a9e0>, <__main__.RRTBranch object at 0x7b4f9867a2c0>, <__main__.RRTBranch object at 0x7b4f98679900>, <__main__.RRTBranch object at 0x7b4f98679a20>, <__main__.RRTBranch object at 0x7b4f9867b640>, <__main__.RRTBranch object at 0x7b4f9845bee0>, <__main__.RRTBranch object at 0x7b4f98

3

 [<__main__.RRTBranch object at 0x7b4f9867a440>, <__main__.RRTBranch object at 0x7b4f98678d90>, <__main__.RRTBranch object at 0x7b4f9867b0a0>, <__main__.RRTBranch object at 0x7b4f9867ae60>, <__main__.RRTBranch object at 0x7b4f9867aec0>, <__main__.RRTBranch object at 0x7b4f9867b160>, <__main__.RRTBranch object at 0x7b4f9867b100>, <__main__.RRTBranch object at 0x7b4f9867b580>, <__main__.RRTBranch object at 0x7b4f986784f0>, <__main__.RRTBranch object at 0x7b4f9867a590>, <__main__.RRTBranch object at 0x7b4f9867b850>, <__main__.RRTBranch object at 0x7b4f9867a530>, <__main__.RRTBranch object at 0x7b4f98679630>, <__main__.RRTBranch object at 0x7b4f98678850>, <__main__.RRTBranch object at 0x7b4f9867a9e0>, <__main__.RRTBranch object at 0x7b4f9867a2c0>, <__main__.RRTBranch object at 0x7b4f98679900>, <__main__.RRTBranch object at 0x7b4f98679a20>, <__main__.RRTBranch object at 0x7b4f9867b640>, <__main__.RRTBranch object at 0x7b4f9845bee0>, <__main__.RRTBranch object at 0x7b4f9845b9d0>, <__main__.R

### CUDA MultiThreading Version *One*




### MultiTherading version two
this version is the one you wanted to implement, parellel stepping only returns a pointif its valid and worker keeps seaching for one until its valid. m is the amount of threads



In [ ]:
import threading


m = 10

count = 0

def parallel_stepping():

  q_rand = trajectory_sample()
  nearest_point = find_closest_point_in_tree(tree, numpy.array(q_rand))
  q_new = step(nearest_point.q, q_rand)
  if is_segment_valid():
    return nearest_point, q_new
  return None

def worker(q):
    while True:
        result = parallel_stepping()
        if result:
            q.put(result)
            return


def find_path():
    for i in range(max_iterations):
      found = False
      q = queue.Queue()
      threads = []

      for _ in range(m):
          t = threading.Thread(target=worker, args=(q,))
          threads.append(t)
          t.start()
      try:
          if not q.empty():
              nearest_point, q_new = q.get()

              if is_segment_valid():
                  new_branch = RRTBranch(parent=nearest_point, q=q_new)
                  tree.append(new_branch)

                  if is_segment_valid():
                      goal_branch = RRTBranch(parent=new_branch, q=q_goal)
                      tree.append(goal_branch)
                      found = True
                      amount_to_sleep = numpy.random.rand(1)[0] * 1
                      time.sleep(amount_to_sleep)
                      print(amount_to_sleep, tree)
                      q.put(tree)
                      return tree

      finally:
          for t in threads:
              t.join()





In [ ]:
len(find_path())

0.4147388184314471 [<__main__.RRTBranch object at 0x7cc296d09900>, <__main__.RRTBranch object at 0x7cc296e5d000>, <__main__.RRTBranch object at 0x7cc296e5f6a0>]


3

C++ Helper Functions

In [ ]:
!nvcc --version

# make sure you have a GPU runtime (if this fails go to runtime -> change runtime type)
!nvidia-smi

# Install some magic to run and save .cu C++ CUDA programs
!curl -o ./gpu_runner.py https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/gpu_runner.py
%load_ext gpu_runner


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
/bin/bash: line 1: nvidia-smi: command not found
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3082  100  3082    0     0   7212      0 --:--:-- --:--:-- --:--:--  7200


In [ ]:
import numpy
import random
import math
import time

num_joints = 3

def trajectory_sample():
    q_rand = []
    for i in range(num_joints):
        q_rand.append(random.uniform(-math.pi, math.pi))
    return q_rand


def find_closest_point_in_tree(tree, r):
        shortest_distance = numpy.linalg.norm(r-tree[0].q)
        closest_point = tree[0]
        for i in range(1, len(tree)-1):
            if shortest_distance > numpy.linalg.norm(r-tree[i].q):
                shortest_distance = numpy.linalg.norm(r-tree[i].q)
                closest_point = tree[i]
        return closest_point


def step(q_near, q_rand, step_size=0.1):
        direction = [qr - qn for qr, qn in zip(q_rand, q_near)]
        length = math.sqrt(sum([d**2 for d in direction]))
        if length == 0:
            return q_near
        step = [qn + (d / length) * step_size for qn, d in zip(q_near, direction)]
        #joint limits
        return [max(min(q, math.pi), -math.pi) for q in step]

def is_segment_valid():
  num = numpy.random.rand(1)
  if num < 0.5:
    return True
  return False

class RRTBranch(object):
    def __init__(self, parent, q):
        self.parent = parent
        self.q = q
